In [14]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import NearMiss
from sklearn.metrics import roc_curve, auc, average_precision_score
from collections import Counter
import time

In [2]:
#independent dataset
indep =pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/indep_descriptor.tsv")
indep_x = indep[indep.columns.values[3:].tolist()].values
indep_y = indep['association'].values

In [3]:
# 기존 데이터 set
df =pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/test_whole_descriptor.tsv")
x_whole_data = df[df.columns.values[3:].tolist()].values
y_whole_data = df['association'].values

In [5]:
def Logist_Regression_independent(x_whole_data, y_whole_data, indep_x, indep_y, user_c):
    #indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_2_indep_asso_descriptor.tsv")
    #indep_x = indep[indep.columns.values[3:].tolist()].values
    #indep_y = indep["association"].values

    classifier = LogisticRegression(penalty='l1', C=user_c, n_jobs = -1)
    #classifier = RandomForestClassifier(n_estimators=user_estimators, n_jobs=-1, class_weight='balanced')
    classifier.fit(x_whole_data, y_whole_data)
    indep_y_predicted_proba = classifier.predict_proba(indep_x)
    indep_y_predicted_label = classifier.predict(indep_x)

    indep_accuracy = classifier.score(indep_x, indep_y)
    indep_fpr, indep_tpr, indep_thresholds = roc_curve(indep_y, indep_y_predicted_proba[:, 1], pos_label = 1)
    indep_auc = auc(indep_fpr, indep_tpr)
    indep_confusion_matirx = confusion_matrix(indep_y, indep_y_predicted_label)
    indep_aupr = average_precision_score(indep_y, indep_y_predicted_proba[:, 1])

    #print("{}----------------------------------------------------".format(user_estimators))
    #print("Independent Accuracy : {}".format(indep_accuracy))
    #print("Independent AUC : {}".format(indep_auc))
    #print("Independent AUPR : {}".format(indep_aupr))
    #print("Independent Confustion matrix")
    #print(indep_confusion_matirx)
    #print("\t")
    
    return [indep_accuracy, indep_auc, indep_aupr, indep_confusion_matirx]
  

In [6]:
def Logist_Regression_independent_10_fold(x_whole_data, y_whole_data, indep_x, indep_y, user_c, undersampler):
    #rus = RandomUnderSampler()
    whole_accuracy = list()
    whole_auc = list()
    whole_aupr = list()
    whole_confusion_mat = list()
    for i in range(0, 10):
        x_resampled, y_resampled = undersampler.fit_sample(indep_x, indep_y)
        indep_result = Logist_Regression_independent(x_whole_data, y_whole_data, x_resampled, y_resampled, user_c)
        whole_accuracy.append(indep_result[0])
        whole_auc.append(indep_result[1])
        whole_aupr.append(indep_result[2])
        whole_confusion_mat.append(indep_result[3])
    print('Original dataset shape {}'.format(Counter(indep_y)))
    print('\nResampled dataset shape {}'.format(Counter(y_resampled)))
    avg_acc = sum(whole_accuracy)/len(whole_accuracy)
    avg_auc = sum(whole_auc)/len(whole_auc)
    avg_aupr = sum(whole_aupr)/len(whole_aupr)
    confusion_matrix = sum(whole_confusion_mat)
    return [avg_acc, avg_auc, avg_aupr, confusion_matrix]

In [7]:
def write_output(filedescritor, results):
    f.write("Accuracy : {}\r\n".format(results[0]))
    f.write("AUC : {}\r\n".format(results[1]))
    f.write("AUPR : {}\r\n".format(results[2]))
    f.write("confusion amtrix : {}\r\n".format(results[3]))

In [8]:
# undersampler
rus = RandomUnderSampler()
oss = OneSidedSelection()
nm = NearMiss()

In [15]:
print time.strftime('%a %H:%M:%S')
with open("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/1_2_original_indep.txt", 'w') as f :
    user_c = 1000
    f.write("<independent>\r\n")
    indep_results = Logist_Regression_independent(x_whole_data, y_whole_data, indep_x, indep_y, user_c)
    write_output(f, indep_results)
    f.write("<independent 1:1>\r\n")
    indep_results_10 = Logist_Regression_independent_10_fold(x_whole_data, y_whole_data, indep_x, indep_y, user_c, rus)
    write_output(f, indep_results_10)
print time.strftime('%a %H:%M:%S')

Tue 17:45:23
Original dataset shape Counter({0: 2396, 1: 89})

Resampled dataset shape Counter({0: 89, 1: 89})
Tue 17:45:42
